In [ ]:
!pip install mp_api
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mp_api.client import MPRester

# Setting API key
API_KEY = ""

def extract_band_gap(materials):
    """Extract band gap values from material entries safely."""
    data = []
    for mat in materials:
        for entry in getattr(mat, "entries", []):  # Correctly access entries
            if hasattr(entry, "band_gap"):
                band_gap = entry.band_gap
                if band_gap is not None:
                    data.append({"Formula": mat.formula_pretty, "Band Gap (eV)": band_gap})

    df = pd.DataFrame(data)

    # Debugging step: Check column names
    print("Columns in extracted DataFrame:", df.columns)

    # Ensure column is correctly labeled inside the function before returning
    if "band_gap" in df.columns:
        df.rename(columns={"band_gap": "Band Gap (eV)"}, inplace=True)

    return df  # Returning the corrected DataFrame

def get_band_gaps():
    """Retrieve band gap data for oxides and nitrides."""
    with MPRester(API_KEY) as mpr:
        oxides = mpr.materials.search(elements=["O"], fields=["material_id", "formula_pretty", "entries"])
        nitrides = mpr.materials.search(elements=["N"], fields=["material_id", "formula_pretty", "entries"])
    return df_oxides, df_nitrides

    df_oxides = extract_band_gap(oxides)
    df_nitrides = extract_band_gap(nitrides)


print("Oxides DataFrame preview:\n", df_oxides.head())
print("Nitrides DataFrame preview:\n", df_nitrides.head())


def visualize_band_gaps(df_oxides, df_nitrides):
    """Plot band gap distributions for oxides and nitrides with safe indexing."""
    plt.figure(figsize=(10, 5))

    # Close previous plots
    plt.close("all")
    if "Band Gap (eV)" in df_oxides.columns:
        plt.hist(df_oxides["Band Gap (eV)"], bins=20, alpha=0.7, label="Oxides", color="blue")

    if "Band Gap (eV)" in df_nitrides.columns:
        plt.hist(df_nitrides["Band Gap (eV)"], bins=20, alpha=0.7, label="Nitrides", color="red")

    plt.xlabel("Band Gap (eV)")
    plt.ylabel("Frequency")
    plt.title("Comparison of Band Gaps: Oxides vs. Nitrides")
    plt.legend()
    plt.show()

# Retrieve and visualize band gaps correctly
df_oxides, df_nitrides = get_band_gaps()
visualize_band_gaps(df_oxides, df_nitrides)